# CS6700: Reinforcement Learning
## Programming Assignment 1

Submitted by:
- Archish S (ME20B032)
- Vinayak Gupta (EE20B152)

# Imports

## Libraries

In [1]:
import numpy as np
from math import floor

## Environment Definition

In [3]:
class BaseEnv:
    
    def step(self, state, action):
        raise NotImplementedError
    
    def reset(self):
        raise NotImplementedError

In [4]:
class GridWorld(BaseEnv):
    """
    Creates a gridworld object to pass to an RL algorithm.
    Parameters
    ----------
    num_rows : int
        The number of rows in the gridworld.
    num_cols : int
        The number of cols in the gridworld.
    start_state : numpy array of shape (1, 2), np.array([[row, col]])
        The start state of the gridworld (can only be one start state)
    goal_states : numpy arrany of shape (n, 2)
        The goal states for the gridworld where n is the number of goal
        states.
    """
    def __init__(self, num_rows, num_cols, start_state, goal_states, wind = False):
        self.num_rows = num_rows
        self.num_cols = num_cols
        self.start_state = start_state
        self.goal_states = goal_states
        self.obs_states = None
        self.bad_states = None
        self.num_bad_states = 0
        self.p_good_trans = None
        self.bias = None
        self.r_step = None
        self.r_goal = None
        self.r_dead = None
        self.gamma = 1 # default is no discounting
        self.wind = wind

    def add_obstructions(self, obstructed_states=None, bad_states=None, restart_states=None):

        self.obs_states = obstructed_states
        self.bad_states = bad_states
        if bad_states is not None:
            self.num_bad_states = bad_states.shape[0]
        else:
            self.num_bad_states = 0
        self.restart_states = restart_states
        if restart_states is not None:
            self.num_restart_states = restart_states.shape[0]
        else:
            self.num_restart_states = 0

    def add_transition_probability(self, p_good_transition, bias):

        self.p_good_trans = p_good_transition
        self.bias = bias

    def add_rewards(self, step_reward, goal_reward, bad_state_reward=None, restart_state_reward = None):

        self.r_step = step_reward
        self.r_goal = goal_reward
        self.r_bad = bad_state_reward
        self.r_restart = restart_state_reward


    def create_gridworld(self):

        self.num_actions = 4
        self.num_states = self.num_cols * self.num_rows# +1
        self.start_state_seq = GridWorld.row_col_to_seq(self.start_state, self.num_cols)
        self.goal_states_seq = GridWorld.row_col_to_seq(self.goal_states, self.num_cols)

        # rewards structure
        self.R = self.r_step * np.ones((self.num_states, 1))
        #self.R[self.num_states-1] = 0
        self.R[self.goal_states_seq] = self.r_goal

        for i in range(self.num_bad_states):
            if self.r_bad is None:
                raise Exception("Bad state specified but no reward is given")
            bad_state = GridWorld.row_col_to_seq(self.bad_states[i,:].reshape(1,-1), self.num_cols)
            #print("bad states", bad_state)
            self.R[bad_state, :] = self.r_bad
        for i in range(self.num_restart_states):
            if self.r_restart is None:
                raise Exception("Restart state specified but no reward is given")
            restart_state = GridWorld.row_col_to_seq(self.restart_states[i,:].reshape(1,-1), self.num_cols)
            #print("restart_state", restart_state)
            self.R[restart_state, :] = self.r_restart

        # probability model
        if self.p_good_trans == None:
            raise Exception("Must assign probability and bias terms via the add_transition_probability method.")

        self.P = np.zeros((self.num_states,self.num_states,self.num_actions))
        for action in range(self.num_actions):
            for state in range(self.num_states):

                # check if the state is the goal state or an obstructed state - transition to end
                row_col = GridWorld.seq_to_col_row(state, self.num_cols)
                if self.obs_states is not None:
                    end_states = np.vstack((self.obs_states, self.goal_states))
                else:
                    end_states = self.goal_states

                if any(np.sum(np.abs(end_states-row_col), 1) == 0):
                    self.P[state, state, action] = 1

                # else consider stochastic effects of action
                else:
                    for dir in range(-1,2,1):

                        direction = self._get_direction(action, dir)
                        next_state = self._get_state(state, direction)
                        if dir == 0:
                            prob = self.p_good_trans
                        elif dir == -1:
                            prob = (1 - self.p_good_trans)*(self.bias)
                        elif dir == 1:
                            prob = (1 - self.p_good_trans)*(1-self.bias)

                        self.P[state, next_state, action] += prob

                # make restart states transition back to the start state with
                # probability 1
                if self.restart_states is not None:
                    if any(np.sum(np.abs(self.restart_states-row_col),1)==0):
                        next_state = GridWorld.row_col_to_seq(self.start_state, self.num_cols)
                        self.P[state,:,:] = 0
                        self.P[state,next_state,:] = 1
        return self

    def _get_direction(self, action, direction):

        left = [2,3,1,0]
        right = [3,2,0,1]
        if direction == 0:
            new_direction = action
        elif direction == -1:
            new_direction = left[action]
        elif direction == 1:
            new_direction = right[action]
        else:
            raise Exception("getDir received an unspecified case")
        return new_direction

    def _get_state(self, state, direction):

        row_change = [-1,1,0,0]
        col_change = [0,0,-1,1]
        row_col = GridWorld.seq_to_col_row(state, self.num_cols)
        row_col[0,0] += row_change[direction]
        row_col[0,1] += col_change[direction]

        # check for invalid states
        if self.obs_states is not None:
            if (np.any(row_col < 0) or
                np.any(row_col[:,0] > self.num_rows-1) or
                np.any(row_col[:,1] > self.num_cols-1) or
                np.any(np.sum(abs(self.obs_states - row_col), 1)==0)):
                next_state = state
            else:
                next_state = GridWorld.row_col_to_seq(row_col, self.num_cols)[0]
        else:
            if (np.any(row_col < 0) or
                np.any(row_col[:,0] > self.num_rows-1) or
                np.any(row_col[:,1] > self.num_cols-1)):
                next_state = state
            else:
                next_state = GridWorld.row_col_to_seq(row_col, self.num_cols)[0]

        return next_state

    def reset(self):
      return int(self.start_state_seq)

    def step(self, state, action):
        p, r = 0, np.random.random()
        for next_state in range(self.num_states):

            p += self.P[state, next_state, action]

            if r <= p:
                break

        if (self.wind and np.random.random() < 0.4):

            arr = self.P[next_state, :, 3]
            next_next = np.where(arr == np.amax(arr))
            next_next = next_next[0][0]
            return next_next, self.R[next_next]
        else:
            return next_state, self.R[next_state]

    @staticmethod
    def row_col_to_seq(row_col, num_cols):
        #Converts state number to row_column format
        return row_col[:,0] * num_cols + row_col[:,1]

    @staticmethod
    def seq_to_col_row(seq, num_cols): 
        #Converts row_column format to state number
        r = floor(seq / num_cols)
        c = seq - r * num_cols
        return np.array([[r, c]])

# Policy

## Action Policy Definitions

In [13]:
class BasePolicy:
    @property
    def name(self):
        raise NotImplementedError

    def select_action(self, state, action_values):
        raise NotImplementedError

### $\varepsilon$-Greedy Policy

The $\varepsilon$-greedy policy defined as
$$
\textrm{next\_action} = \begin{cases}
    \underset{a \in A(s)}{\arg\max} \; Q(s, a) & \text{with probability } 1 - \varepsilon \\
    \textrm{random choice} & \text{with probability } \varepsilon
\end{cases}
$$.

Hyperparameters:
- $\varepsilon$: The probability of choosing a random action

In [12]:
class EpGreedyPolicy(BasePolicy):
    @property
    def name(self):
        return f'ep-greedy ep:{self.epsilon}'

    def __init__(self, epsilon, actions):
        self.epsilon = epsilon
        self.actions = actions

    def select_action(self, state, action_values):

        if np.random.binomial(1, 1-self.epsilon):
            return self.actions[np.argmax(action_values[state, :])]
        else:
            return np.random.choice(self.actions)

### Softmax Policy

The softmax policy is defined as
$$
\textrm{next\_action} = \begin{cases}
    a_1 & \text{with probability } \mathcal{P}(1) \\
    a_2 & \text{with probability } \mathcal{P}(2) \\
    \vdots  & \vdots \\
    a_n & \text{with probability } \mathcal{P}(n)
\end{cases}
$$
where 
$$
\mathcal{P}(a) = \dfrac{e^{Q(s, a) / \tau}}{\sum\limits_{i=1}^{n} e^{Q(s, i) / \tau}}
$$.

Hyperparameters:
- $\tau$: The temperature parameter

In [8]:
class SoftmaxPolicy(BasePolicy):
    @property
    def name(self):
        return f'softmax tau:{self.temperature}'

    def __init__(self, tau, actions):
        self.tau = tau
        self.actions = actions

    def select_action(self, state, action_values):
        action_probs = np.exp(action_values[state, :] / self.tau)
        action_probs /= np.sum(action_probs)
        return np.random.choice(self.actions, p=action_probs)

### Greedy Policy

In [10]:
class GreedyPolicy(BasePolicy):
    @property
    def name(self):
        return 'greedy'

    def __init__(self, actions):
        self.actions = actions

    def select_action(self, state, action_values):
        return self.actions[np.argmax(action_values[state, :])]

## Policy Update

In [11]:
class BaseUpdate:
    @property
    def name(self):
        raise NotImplementedError
    
    def update(self, state):
        raise NotImplementedError

### SARSA

The update rule for SARSA:
$$
Q(s_t,a_t) \leftarrow  Q(s_t, a_t) + \alpha[r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)]
$$

Hyperparameters:
- $\alpha$: The learning rate
- $\gamma$: The discount factor


In [18]:
class SARSAUpdate(BaseUpdate):
    @property
    def name(self):
        return 'sarsa'

    def __init__(self, env, alpha, gamma, policy):
        self.alpha = alpha
        self.gamma = gamma
        self.env = env
        self.policy = policy

    def update(self, Q, state, action, reward, next_state, next_action):
        return Q[state, action] + self.alpha * (reward + self.gamma * Q[next_state, next_action] - Q[state, action])

### Q-Learning

The update rule for Q-Learning:
$$
Q(s_t,a_t) \leftarrow  Q(s_t, a_t) + \alpha[r_t + \gamma \max_{a} Q(s_{t+1}, a) - Q(s_t, a_t)]
$$

Hyperparameters:
- $\alpha$: The learning rate
- $\gamma$: The discount factor

In [17]:
class QLearningUpdate(BaseUpdate):
    @property
    def name(self):
        return 'q-learning'

    def __init__(self, env, alpha, gamma, policy):
        self.alpha = alpha
        self.gamma = gamma
        self.env = env
        self.policy = policy

    def update(self, Q, state, action, next_state, next_action, reward):
        return Q[state, action] + self.alpha * (reward + self.gamma * np.max(Q[next_state, :]) - Q[state, action])